### 초기설정

In [1]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=65d82253a24127311f5b0984148ebd0b8391dc15ecec2af9992f0b480c41800c
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
!pip install umap-learn

     |████████████████████████████████| 80 kB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 23.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-py3-none-any.whl size=76566 sha256=f66238ae4353bdff098b74726ed92541291d025953082e19dbe5cd76a9260b9b
  Stored in directory: /root/.cache/pip/wheels/01/e7/bb/347dc0e510803d7116a13d592b10cc68262da56a8eec4dd72f
  Created wheel for pynndescent: filename=pynndescent-0.5.4-py3-none-any.whl size=52372 sha256=ceec07c60fbba50fd9738888608a79659dd6d647c81238f5462eed7598acac82
  Stored in directory: /root/.cache/pip/wheels/d0/5b/62/3401692ddad12324249c774c4b15ccb046946021e2b581c043
Successfully built umap-learn pynndescent


In [2]:
from efficientnet_pytorch import EfficientNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# from keras.applications.vgg16 import VGG16 
# from keras.models import Model
# from keras.applications.vgg16 import preprocess_input 
import os
import logging
import torch
from torch import nn
import pickle
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
#dataset path
data_path = '/content/drive/MyDrive/Colab/Sketch_RNN_Together/Dataset/test/'
#feature path
feat_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/features_50_2000.npy"
#filename path
filename_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/filenames_50_2000.npy"
#cluster_list
cluster_list_path = '/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/cluster_list.txt'
cluster_list = open(cluster_list_path , 'r').read().split('\n')

# p = r"/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/sktch_log.pkl"

In [4]:
#make data_list : .npy files
os.chdir(data_path)
data_list = []
with os.scandir(data_path) as files:
    for file in files:
        if file.name.endswith('.npy'):
            data_list.append(file.name) 

### Feature Extraction

In [ ]:
#transform
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

def feature_extraction(image, model):
    img = tfms(Image.fromarray(image.reshape(28,28)).convert("RGB")).unsqueeze(0)
    f = model.extract_features(img)
    ap = nn.AdaptiveAvgPool2d(1)
    features = ap(f)
    return features

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b0')
data = {}
cluster_list = []
for npy in tqdm(data_list[:50]):
    cluster_list.append(npy.split('.')[0])
    image_list = np.load(npy) 
    for i, image in enumerate(image_list[:2000]):
        feat = feature_extraction(image, model)
        feat = feat.detach().numpy().reshape(-1)
        name = npy.split('.')[0] + '_' + str(i)
        data[name] = (feat)
        if(i % 500 == 0):
          np.save(feat_path, np.array(list(data.values())))
          np.save(filename_path, np.array(list(data.keys())))

feat = np.array(list(data.values()))
filenames = np.array(list(data.keys()))
np.save(feat_path, np.array(list(data.values())))
np.save(filename_path, np.array(list(data.keys())))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0/50 [00:00<?, ?it/s]


Loaded pretrained weights for efficientnet-b0


100%|██████████| 50/50 [2:16:56<00:00, 164.33s/it]


In [ ]:
with open(cluster_list_path, 'w') as file:
  for l in cluster_list:
    file.write(l + '\n')

### Dimension Reduction

In [ ]:
#PCA
# feat = np.load(feat_path)
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2, random_state=0)
# pca.fit(feat)
# x = pca.transform(feat)

In [ ]:
#TSNE
feat = np.load(feat_path)
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, init='pca', random_state=0)
x = tsne.fit_transform(feat)

In [ ]:
#UMAP
# feat = np.load(feat_path)
# import umap
# umap = umap.UMAP(n_components=2, random_state=0)
# umap.fit(feat)
# x = umap.transform(feat)

### Clustering

In [ ]:
def clustering_method(cmd, cluster_num, input):
  if cmd == 'kmeans':
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=cluster_num, random_state = 0)
    kmeans.fit(input)
    return kmeans.labels_

  elif cmd == 'minibatch':
    from sklearn.cluster import MiniBatchKMeans
    mb_kmeans = MiniBatchKMeans(n_clusters=cluster_num,random_state=0, batch_size=6)
    mb_kmeans.fit(input)
    return mb_kmeans.labels_

  elif cmd == 'dbscan':
    from sklearn.cluster import DBSCAN
    dbscan = DBSCAN(eps=1.3, min_samples=2)
    dbscan.fit(input)
    return dbscan.labels_

  elif cmd == 'birch':
    from sklearn.cluster import Birch
    brc = Birch(threshold=0.3, branching_factor=5, n_clusters=10)
    brc.fit(input)
    return brc.labels_

In [ ]:
method = 'dbscan'
labels = clustering_method(method, len(cluster_list), x)
filenames = np.load(filename_path)
groups = {}
# holds the cluster id and the images { id: [images] }
for f, cluster in zip(filenames, labels):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(f.split('_')[-2])
    else:
        groups[cluster].append(f.split('_')[-2])

In [ ]:
#make cluster_dict for calculate acc
cluster_dict = {}
for cluster in groups:
  image_count = []
  image_name = []
  for image in groups[cluster]:
    image_name.append(image)
  for name in cluster_list:
    image_count.append(image_name.count(name))
  cluster_dict[cluster] = cluster_list[image_count.index(max(image_count))]

#Acc
from sklearn.metrics import f1_score
pred = []
gt = []    
for cluster in groups:
    for draw in groups[cluster]:
        pred.append(cluster_dict[cluster])
        gt.append(draw)

import plotly.express as px
from plotly.offline import plot
fig = px.scatter(x, x=0, y=1, color=labels)
fig.show()
print("F1 ACC: " + str(f1_score(gt, pred, average='micro') * 100))

F1 ACC: 69.91199999999999
